# TEMIS UV Dose Data — Download & Monthly Aggregation

In [1]:
from pathlib import Path
import urllib.request
import xarray as xr
import numpy as np

## Download daily NetCDF

In [2]:
DATA_DIR = Path(".")
NC_PATH = DATA_DIR / "uvdvcclim_world.nc"

URL = "https://d1qb6yzwaaq4he.cloudfront.net/uvradiation/v2.0/nc/clim/uvdvcclim_world.nc"

if NC_PATH.exists():
    print(f"File already exists: {NC_PATH}")
else:
    print(f"Downloading to {NC_PATH} ...")
    urllib.request.urlretrieve(URL, NC_PATH)
    print("Done.")

File already exists: uvdvcclim_world.nc


## Load daily data

In [3]:
ds = xr.open_dataset(NC_PATH, group="PRODUCT")
ds

<xarray.Dataset>
Dimensions:            (latitude: 720, longitude: 1440, corner: 4, days: 365)
Coordinates:
  * latitude           (latitude) float32 -89.88 -89.62 -89.38 ... 89.62 89.88
  * longitude          (longitude) float32 -179.9 -179.6 -179.4 ... 179.6 179.9
  * corner             (corner) float32 0.0 1.0 2.0 3.0
  * days               (days) float64 1.0 2.0 3.0 4.0 ... 363.0 364.0 365.0
Data variables: (12/15)
    latitude_index     (latitude) float64 ...
    longitude_index    (longitude) float64 ...
    latitude_bounds    (latitude, corner) float32 ...
    longitude_bounds   (longitude, corner) float32 ...
    date               (days) float64 ...
    date_string        (days) object ...
    ...                 ...
    uvd_clear_min      (days, latitude, longitude) float32 ...
    uvd_clear_max      (days, latitude, longitude) float32 ...
    uvd_cloudy_mean    (days, latitude, longitude) float32 ...
    uvd_cloudy_stddev  (days, latitude, longitude) float32 ...
    uvd_cloudy_min     (days, latitude, longitude) float32 ...
    uvd_cloudy_max     (days, latitude, longitude) float32 ...

## Aggregate daily to monthly means

Map each day-of-year (1–365) to its calendar month and compute the mean over all days in each month.

In [8]:
ds_monthly = ds.groupby(ds["date_month"].rename("month")).mean(dim="days")
ds_monthly

<xarray.Dataset>
Dimensions:            (month: 12, latitude: 720, longitude: 1440, corner: 4)
Coordinates:
  * latitude           (latitude) float32 -89.88 -89.62 -89.38 ... 89.62 89.88
  * longitude          (longitude) float32 -179.9 -179.6 -179.4 ... 179.6 179.9
  * corner             (corner) float32 0.0 1.0 2.0 3.0
  * month              (month) int32 1 2 3 4 5 6 7 8 9 10 11 12
Data variables: (12/14)
    date               (month) float64 116.0 214.5 316.0 ... 1.116e+03 1.216e+03
    date_month         (month) float64 1.0 2.0 3.0 4.0 ... 9.0 10.0 11.0 12.0
    uvd_clear_mean     (month, latitude, longitude) float32 3.755 3.755 ... 0.0
    uvd_clear_stddev   (month, latitude, longitude) float32 0.2884 ... 0.0
    uvd_clear_min      (month, latitude, longitude) float32 3.339 3.342 ... 0.0
    uvd_clear_max      (month, latitude, longitude) float32 4.43 4.425 ... 0.0
    ...                 ...
    uvd_cloudy_min     (month, latitude, longitude) float32 nan nan ... nan nan
    uvd_cloudy_max     (month, latitude, longitude) float32 nan nan ... nan nan
    latitude_index     (month, latitude) float64 0.0 1.0 2.0 ... 718.0 719.0
    longitude_index    (month, longitude) float64 0.0 1.0 ... 1.439e+03
    latitude_bounds    (month, latitude, corner) float32 -90.0 -89.75 ... 89.75
    longitude_bounds   (month, longitude, corner) float32 -179.8 ... 179.8

## Save monthly data

In [10]:
OUT_PATH = DATA_DIR / "uvdvcclim_world_monthly.nc"
ds_monthly.to_netcdf(OUT_PATH)
print(f"Saved monthly data to {OUT_PATH}")

Saved monthly data to uvdvcclim_world_monthly.nc
